In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


##Creating New Features

### Use: Age, Fare, SibSp, Parch

###New features: FamilySize,FarePerPerson, AgeGroup

In [4]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['FarePerPerson'] = df['Fare'] / df['FamilySize']

In [5]:
df['AgeGroup'] = pd.cut(
    df['Age'],
    bins=[0, 12, 18, 35, 60, 100],
    labels=['Child', 'Teen', 'Adult', 'MidAge', 'Senior']
)

##Handling Date & Text Features

In [6]:
df['NameLength'] = df['Name'].apply(len)

In [7]:
df['NameWordCount'] = df['Name'].apply(lambda x: len(x.split()))

##Feature Encoding

###One-Hot Encoding (Using pandas)

In [8]:
categorical_cols = ['Sex', 'Embarked', 'AgeGroup']

df_dummies = pd.get_dummies(df[categorical_cols], drop_first=True)
df_dummies.head()

,Sex_male,Embarked_Q,Embarked_S,AgeGroup_Teen,AgeGroup_Adult,AgeGroup_MidAge,AgeGroup_Senior
0,True,False,True,False,True,False,False
1,False,False,False,False,False,True,False
2,False,False,True,False,True,False,False
3,False,False,True,False,True,False,False
4,True,False,True,False,True,False,False


###ColumnTransformer

In [9]:
numerical_features = [
    'Age', 'Fare', 'FamilySize',
    'FarePerPerson', 'NameLength', 'NameWordCount'
]

categorical_features = ['Sex', 'Embarked', 'AgeGroup']

In [10]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

In [11]:
X_transformed = preprocessor.fit_transform(df)

##Feature Scaling

###Standardization (StandardScaler)

In [12]:
scaler = StandardScaler()
scaled_standard = scaler.fit_transform(df[numerical_features])

scaled_standard.mean(axis=0)
scaled_standard.std(axis=0)

array([nan,  1.,  1.,  1.,  1.,  1.])

###Normalization (MinMaxScaler)

In [13]:
minmax = MinMaxScaler()
scaled_minmax = minmax.fit_transform(df[numerical_features])

scaled_minmax.min(axis=0)
scaled_minmax.max(axis=0)

array([nan,  1.,  1.,  1.,  1.,  1.])

##Building ML Pipeline

###Preprocessing Pipelines

In [25]:
numeric_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [26]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
        ]
)

In [27]:
X = df[numerical_features + categorical_features]
y = df['Survived']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [33]:
X_train = X_train.dropna()
X_test= X_test.dropna()
y_train = y_train[X_train.index]

In [34]:
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', LogisticRegression(max_iter=1000))
])

In [35]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare', 'FamilySize',
                                                   'FarePerPerson',
                                                   'NameLength',
                                                   'NameWordCount']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'AgeGroup'])])),
                ('model', LogisticRegression(max_iter=1000))])

In [36]:
y_pred = model_pipeline.predict(X_test)
y_pred[:10]

array([0, 0, 1, 1, 1, 0, 1, 1, 0, 0])